<a href="https://colab.research.google.com/github/CarlaLS/Entregas-CoderHouse-Ciencia-de-Datos-III/blob/main/Proyecto_Final_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Análisis de Sentimientos en Tweets utilizando Machine Learning


## Introducción
En la era digital, plataformas como Twitter generan un flujo constante de opiniones y sentimientos de los usuarios. Comprender estas opiniones es esencial para empresas, investigadores y organizaciones, ya que les permite medir la percepción pública, tomar decisiones informadas y ajustar sus estrategias de marketing o servicio al cliente. Este proyecto se centra en el análisis de sentimientos en tweets, utilizando técnicas de Procesamiento de Lenguaje Natural (NLP) y modelos de Machine Learning para clasificar los sentimientos en tres categorías: positivo, negativo y neutral.

## Objetivos
**Objetivo General**

Desarrollar un modelo de Machine Learning eficaz para clasificar tweets en tres categorías de sentimientos: positivo, negativo y neutral.

**Objetivos Específicos**

Realizar un análisis exploratorio de datos para entender las características del conjunto de datos.

Diseñar y aplicar un pipeline de preprocesamiento para limpiar y normalizar el texto.

Entrenar varios modelos de Machine Learning para clasificar los sentimientos en los tweets.

Implementar una evaluación exhaustiva de los modelos para identificar el más eficiente.

Validar el modelo seleccionado mediante predicciones en nuevos ejemplos y analizar su rendimiento.

## Metodología

**Dataset**

Se utilizó el dataset disponible en el repositorio de GitHub: "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv". Este dataset contiene tweets clasificados originalmente en etiquetas binarias (positivo y negativo), que fueron reclasificados en tres categorías: positivo, negativo y neutral.

In [10]:
# Liberías Necesarias

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from imblearn.over_sampling import SMOTE

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
# 1. Carga de datos
data_url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
data = pd.read_csv(data_url)

# Primeras filas del dataset
print("Dataset original:")
print(data.head())

Dataset original:
   id  label                                              tweet
0   1      0   @user when a father is dysfunctional and is s...
1   2      0  @user @user thanks for #lyft credit i can't us...
2   3      0                                bihday your majesty
3   4      0  #model   i love u take with u all the time in ...
4   5      0             factsguide: society now    #motivation


**Procesamiento de Datos**

*Limpieza de Datos:*

Se eliminaron enlaces, menciones, caracteres especiales y stopwords irrelevantes.

*Reclasificación de Sentimientos:*

Tweets con palabras clave positivas (“love”, “amazing”) fueron etiquetados como positivos.

Tweets con palabras clave negativas (“worst”, “bad”) fueron etiquetados como negativos.

Los tweets que no contenían palabras clave específicas se clasificaron como neutrales.

*Vectorización*:

Se utilizó TfidfVectorizer para convertir el texto en representaciones numéricas.

**Modelos Implementados**

*Naive Bayes Multinomial:*

 Un modelo base conocido por su eficiencia en tareas de clasificación de texto.

*Regresión Logística:*

Optimizada con GridSearchCV para encontrar los mejores hiperparámetros.

*Random Forest:*

Utilizado para capturar relaciones no lineales entre las características.

**Manejo del Desbalanceo de Clases**

Se aplicó SMOTE (Synthetic Minority Oversampling Technique) para balancear las clases durante el entrenamiento.

In [3]:
# 2. Exploración y limpieza del dataset
# Solo columnas necesarias: texto y sentimiento
data = data[["label", "tweet"]]
data = data.rename(columns={"label": "sentiment", "tweet": "text"})

In [4]:
# 3. Verificar valores nulos
data.dropna(inplace=True)

# Reclasificación para incluir la clase neutral
def reclasificar_sentimiento(row):
    positive_keywords = ["love", "great", "amazing", "excellent", "good"]
    negative_keywords = ["worst", "bad", "awful", "terrible", "hate"]

    text = row["text"].lower()

    if any(word in text for word in positive_keywords):
        return 1  # Positivo
    elif any(word in text for word in negative_keywords):
        return 0  # Negativo
    else:
        return 2  # Neutral

data["sentiment"] = data.apply(reclasificar_sentimiento, axis=1)

# Mostrar distribución de sentimientos
print("\nDistribución de sentimientos:")
print(data["sentiment"].value_counts())



Distribución de sentimientos:
sentiment
2    25606
1     5561
0      795
Name: count, dtype: int64


In [5]:
# 4. Preprocesamiento del texto
def clean_text_advanced(text):
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))

    # Eliminar enlaces, menciones y caracteres especiales
    text = re.sub(r'http\S+', '', text)  # Eliminar enlaces
    text = re.sub(r'@\w+', '', text)     # Eliminar menciones
    text = re.sub(r'[^A-Za-z\s]', '', text)  # Eliminar caracteres especiales

    # Convertir a minúsculas
    text = text.lower()

    # Tokenizar y lematizar palabras, excluyendo stopwords
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]

    return ' '.join(tokens)

In [6]:
#5. Aplicar limpieza
X = data["text"].apply(clean_text_advanced)
y = data["sentiment"]

# Dividir datos en entrenamiento y prueba
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)


In [7]:
#6. Vectorización del texto
vectorizer = TfidfVectorizer(max_features=5000)
X_train_vec = vectorizer.fit_transform(X_train_raw)
X_test_vec = vectorizer.transform(X_test_raw)

# 7. Aplicar SMOTE para balancear clases
smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train_vec, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.ClassifierMixin`, `sklearn.base.RegressorMixin`, and `sklearn.base.OutlierMixin`. From scikit-learn 1.7, not defining `__sklearn_tags__` will raise an error.
  warnings.warn(


In [8]:
# 8. Modelos de Machine Learning
# a. Naive Bayes
nb_model = MultinomialNB()
nb_model.fit(X_train_vec, y_train)
nb_preds = nb_model.predict(X_test_vec)
print("\nNaive Bayes:")
print(classification_report(y_test, nb_preds))





Naive Bayes:
              precision    recall  f1-score   support

           0       0.23      0.71      0.35       159
           1       0.54      0.85      0.66      1112
           2       0.96      0.78      0.86      5122

    accuracy                           0.79      6393
   macro avg       0.58      0.78      0.62      6393
weighted avg       0.87      0.79      0.81      6393



In [11]:

# b. Logistic Regression con GridSearchCV
param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'solver': ['liblinear', 'saga']
}
grid = GridSearchCV(LogisticRegression(max_iter=1000, class_weight='balanced', multi_class='ovr'), param_grid, cv=3, scoring='f1_macro')
grid.fit(X_train_vec, y_train)

# Mejor modelo Logistic Regression
best_lr = grid.best_estimator_
print("\nMejores hiperparámetros Logistic Regression:", grid.best_params_)

lr_preds = best_lr.predict(X_test_vec)
print("\nLogistic Regression:")
print(classification_report(y_test, lr_preds))



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:1256: FutureWarning: 'multi_class' was depre


Mejores hiperparámetros Logistic Regression: {'C': 10, 'solver': 'liblinear'}

Logistic Regression:
              precision    recall  f1-score   support

           0       0.69      0.84      0.76       159
           1       0.85      0.89      0.87      1112
           2       0.97      0.96      0.96      5122

    accuracy                           0.94      6393
   macro avg       0.84      0.90      0.86      6393
weighted avg       0.94      0.94      0.94      6393



In [12]:
# c. Random Forest
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train_vec, y_train)
rf_preds = rf_model.predict(X_test_vec)
print("\nRandom Forest:")
print(classification_report(y_test, rf_preds))


Random Forest:
              precision    recall  f1-score   support

           0       0.84      0.84      0.84       159
           1       0.91      0.88      0.90      1112
           2       0.97      0.98      0.97      5122

    accuracy                           0.96      6393
   macro avg       0.91      0.90      0.90      6393
weighted avg       0.96      0.96      0.96      6393



## Observaciones

**Naive Bayes**

*Desbalance de clases:*  La clase 2 domina el dataset, lo que hace que el modelo priorice su predicción, penalizando el desempeño en las clases 0 y 1.

*Clase 0*:  El modelo tiene serias dificultades para identificar correctamente esta clase (baja precisión y F1-score).

*Clase 1:* Aunque tiene un desempeño aceptable, aún hay margen para mejorar su precisión.

**Logistic Regression**

*Clase 0:* El modelo tiene un rendimiento aceptable considerando que es la clase minoritaria. Sin embargo, la precisión moderada (0.69) indica que aún se confunde con las otras clases en ciertos casos.

*Clase 1:* El modelo maneja esta clase con alta precisión y recall, mostrando un equilibrio sólido en su clasificación.

*Clase 2:* Como clase dominante en el dataset, el modelo sobresale con casi perfecta precisión y recall, lo que resulta en un excelente F1-score (0.96).

**Random Forest**

*Clase 0:* A pesar de ser minoritaria, el modelo la identifica con alta precisión y recall, lo que sugiere que Random Forest maneja bien el desbalance de clases.

*Clase 1:* Consistencia en el rendimiento con un balance sólido entre precisión y recall.

*Clase 2:*  Dominante en el dataset, el modelo sobresale al clasificarla con casi perfecta precisión y recall.


In [13]:
# 9. Comparación de modelos
print("\nAccuracy Scores:")
print(f"Naive Bayes: {accuracy_score(y_test, nb_preds):.2f}")
print(f"Logistic Regression: {accuracy_score(y_test, lr_preds):.2f}")
print(f"Random Forest: {accuracy_score(y_test, rf_preds):.2f}")




Accuracy Scores:
Naive Bayes: 0.79
Logistic Regression: 0.94
Random Forest: 0.96


**Comentarios**

Random Forest tiene el mejor desempeño en términos de accuracy y métricas F1 para todas las clases, manejando bien tanto las mayoritarias como las minoritarias.

Comparado con Logistic Regression, tiene una ligera ventaja, especialmente en la precisión de las clases minoritarias.

Supera significativamente a Naive Bayes, especialmente en la clase 0, donde Naive Bayes tiene dificultades.

 Por su excelente equilibrio entre precisión, recall y F1-score Random Forest se convierte en la mejor opción.

In [14]:
# 10. Desafío del modelo
best_model = best_lr
new_tweets = [
    "I love this product! It's amazing.",
    "This is the worst experience I've ever had.",
    "I'm not sure how I feel about this."
]
new_tweets_cleaned = [clean_text_advanced(tweet) for tweet in new_tweets]
new_tweets_vec = vectorizer.transform(new_tweets_cleaned)
new_preds = best_model.predict(new_tweets_vec)

print("\nPredicciones para nuevos ejemplos:")
for tweet, pred in zip(new_tweets, new_preds):
    sentiment = "Neutral" if pred == 2 else "Positivo" if pred == 1 else "Negativo"
    print(f"Tweet: {tweet}\nPredicción: {sentiment}\n")




Predicciones para nuevos ejemplos:
Tweet: I love this product! It's amazing.
Predicción: Positivo

Tweet: This is the worst experience I've ever had.
Predicción: Negativo

Tweet: I'm not sure how I feel about this.
Predicción: Neutral



## Análisis de las Predicciones para Nuevos Ejemplos

1. Ejemplo: "I love this product! It's amazing."

**Predicción: Positivo**

La predicción es consistente con el sentimiento del tweet, ya que las palabras clave como "love" y "amazing" son fuertemente asociadas con emociones positivas.
La precisión del modelo para identificar textos positivos parece ser sólida, asumiendo que se basa en las características léxicas y el tono emocional del tweet.Este resultado respalda la capacidad del modelo para identificar sentimientos explícitos positivos.


2. Ejemplo: "This is the worst experience I've ever had."

**Predicción: Negativo**

La predicción es correcta, ya que términos como "worst" y "experience" en este contexto tienen una connotación clara de insatisfacción y descontento.
El modelo demuestra un buen manejo del sentimiento negativo explícito. Este tweet representa un ejemplo típico y fácil de clasificar debido a su lenguaje explícito.


3. Ejemplo: "I'm not sure how I feel about this."

**Predicción: Neutral**

La predicción es razonable. El lenguaje del tweet no contiene palabras explícitamente positivas o negativas, y sugiere ambigüedad emocional ("not sure", "how I feel").
La capacidad del modelo para identificar sentimientos neutros parece estar funcionando correctamente, basándose en el contexto y la falta de términos emocionalmente cargados.

# Conclusiones

##Textos explícitos:
Muestra precisión al identificar emociones claras, como en los ejemplos positivo y negativo.

## Ambigüedad:

La predicción neutral sugiere que el modelo maneja correctamente casos donde no hay polaridad evidente.